<a href="https://colab.research.google.com/github/VaLeRiEe37/AIPolicyChatbot/blob/main/Prompt_Engineering_Control_Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate datasets
import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import Dataset, DatasetDict, load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00


## **Load dataset from HuggingFace**

In [11]:
# Dataset
data_name = "jiuyuan/policy_AI"
training_data = load_dataset(data_name, split="train")
test_data = load_dataset(data_name, split="test")

In [3]:
df_test = test_data.to_pandas()
df_test

,input,output
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express..."
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro..."
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult..."
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language..."
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...


In [4]:
df_test[['input']].to_csv('prompts.csv', )

In [6]:
import pandas as pd
df_prompt = pd.read_csv("control_codes_prompts.csv", encoding='ISO-8859-1')
df_prompt['prompt_engineering'] = df_prompt['prompt_engineering']
df_prompt

,input,prompt_engineering
0,Why is international cooperation on AI important?,[International Cooperation] Why is internation...
1,Can you describe the approach taken by the res...,[Research Approach] Can you describe the appro...
2,What concerns do critics have regarding the EU...,[EU AI Act Criticism] What concerns do critics...
3,How does the European Union classify AI system...,[EU AI Classification] How does the European U...
4,What are the Universal Guidelines for Artifici...,[AI Ethics Guidelines] What are the Universal ...
5,What role do whistleblowers and complaints pla...,[Regulatory Process] What role do whistleblowe...
6,How's UK's approach different from that of the...,[UK AI Strategy] How's UK's approach different...
7,How does Japan's approach to AI regulation dif...,[Japan AI Regulation] How does Japan's approac...
8,"What are foundational AI models, and why are t...",[Foundational AI Models] What are foundational...
9,What additional measures are needed for intern...,[AI Cooperation Measures] What additional meas...


In [7]:
def prompt_engineering(df):
    output_texts = []

    for i in range(len(df['input'])):
        row = df.iloc[i]
        text = f"### Question: {row['input']}\n Use this template to answer the question: {row['prompt_engineering']} \n ### Answer: </s>"
        output_texts.append(text)
    return output_texts

df_prompt['prompt']= prompt_engineering(df_prompt)
print(df_prompt.iloc[0]['prompt'])
df_prompt

### Question: Why is international cooperation on AI important?
 Use this template to answer the question: [International Cooperation] Why is international cooperation on AI important? 
 ### Answer: </s>


,input,prompt_engineering,prompt
0,Why is international cooperation on AI important?,[International Cooperation] Why is internation...,### Question: Why is international cooperation...
1,Can you describe the approach taken by the res...,[Research Approach] Can you describe the appro...,### Question: Can you describe the approach ta...
2,What concerns do critics have regarding the EU...,[EU AI Act Criticism] What concerns do critics...,### Question: What concerns do critics have re...
3,How does the European Union classify AI system...,[EU AI Classification] How does the European U...,### Question: How does the European Union clas...
4,What are the Universal Guidelines for Artifici...,[AI Ethics Guidelines] What are the Universal ...,### Question: What are the Universal Guideline...
5,What role do whistleblowers and complaints pla...,[Regulatory Process] What role do whistleblowe...,### Question: What role do whistleblowers and ...
6,How's UK's approach different from that of the...,[UK AI Strategy] How's UK's approach different...,### Question: How's UK's approach different fr...
7,How does Japan's approach to AI regulation dif...,[Japan AI Regulation] How does Japan's approac...,### Question: How does Japan's approach to AI ...
8,"What are foundational AI models, and why are t...",[Foundational AI Models] What are foundational...,"### Question: What are foundational AI models,..."
9,What additional measures are needed for intern...,[AI Cooperation Measures] What additional meas...,### Question: What additional measures are nee...


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-mlabonne-enhanced"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

## **Generations & Evaluation**

In [12]:
df_test_all=test_data.to_pandas()
df_train=training_data.to_pandas()
df_test_all

,input,output
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express..."
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro..."
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult..."
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language..."
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...


In [13]:
df_test_all['output'].apply(lambda x:len(x.split())).describe(), df_train['output'].apply(lambda x:len(x.split())).describe()

(count    19.000000
 mean     40.631579
 std      11.591235
 min      17.000000
 25%      33.000000
 50%      41.000000
 75%      48.500000
 max      61.000000
 Name: output, dtype: float64,
 count      75.000000
 mean       70.000000
 std       135.589205
 min        17.000000
 25%        30.000000
 50%        39.000000
 75%        52.000000
 max      1020.000000
 Name: output, dtype: float64)

In [14]:
from tqdm.notebook import tqdm
import gc
generations_prompt_engineer_controlcodes =[]
for i in tqdm(range(len(df_prompt)), "generating..."):
  prompt = df_prompt.iloc[i]['prompt']
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = base_model.generate(**inputs, max_new_tokens=200,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  print(response)
  # break
  generations_prompt_engineer_controlcodes.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()


generating...:   0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s> ### Question: Why is international cooperation on AI important?
 Use this template to answer the question: [International Cooperation] Why is international cooperation on AI important? 
 ### Answer:</s><s> nobody knows the answer to this question.

International cooperation on AI is important for several reasons:

1. **Sharing knowledge and resources**: By working together, countries can share their expertise and resources in AI, leading to faster progress and more innovative solutions.
2. **Addressing ethical and social implications**: AI raises complex ethical and social implications, such as privacy, bias, and job displacement. International cooperation can help address these issues in a coordinated manner.
3. **Fostering global standards**: Establishing common standards and guidelines for AI can help ensure that AI is developed and used in a responsible and secure manner.
4. **Promoting economic growth**: AI has the potential to drive economic growth and create new opportunitie

In [15]:
df_test['prompt_engineering_control_codes_generations']=generations_prompt_engineer_controlcodes
df_test

,input,output,prompt_engineering_control_codes_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,<s> ### Question: What additional measures are...


In [22]:
# Model
raw_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
raw_model.config.use_cache = False
raw_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from tqdm.notebook import tqdm
import gc
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = raw_model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations_original

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

['<s> ### Question: Why is international cooperation on AI important?\n ### Answer:  International cooperation on AI is important for several reasons:\n\n1. **Sharing knowledge and expertise**: By working together, countries can pool their resources and expertise to advance the field of AI more quickly and effectively. This can lead to the development of new technologies and applications that benefit society as a whole.\n2. **Addressing ethical and social implications**: AI raises a range of ethical and social implications, such as privacy',
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n ### Answer:  The research team took a multi-faceted approach in preparing for the interviews with academia and industry experts in the AI assessment. Here are some of the key steps we took:\n\n1. Literature Review: We conducted a comprehensive literature review to identify the key th

In [24]:
df_test_all['original_llama_generations']=generations_original
df_test_all

,input,output,prompt_engineering_control_codes_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,nobody knowthe answer to thiquestion.Internat...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,"sure, here'how the research team prepared for ...",<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",t iimportant to note that the EU'AI Act istill...,<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,nobody likea know-it-allThe European Union ha...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,...The Electronic Privacy Information Center (...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,sWhistleblowerand complaintplay a crucial role...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",sThe UK'approach to AI idifferent from that of...,<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",nobodyJapan'approach to AI regulation differf...,<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",nobody knowthe answer to thiquestion.Foundati...,<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,nobody knowthe answer to thiquestion yet.Inte...,<s> ### Question: What additional measures are...


In [25]:
df_test_all = df_test
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', '').replace('<s/>', '').replace('s ', ''))
df_test_all['prompt_engineering_control_codes_generations']= df_test_all['prompt_engineering_control_codes_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['prompt_engineering_control_codes_generations']= df_test_all['prompt_engineering_control_codes_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', '').replace('<s/>', '').replace('s ', '').replace('\n', ''))
df_test_all

,input,output,prompt_engineering_control_codes_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,nobody knowthe answer to thiquestion.Internat...,International cooperation on AI iimportant f...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,"sure, here'how the research team prepared for ...",The research team took a multi-faceted appro...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",t iimportant to note that the EU'AI Act istill...,\nCriticof the EU'AI Act have raised several ...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,nobody likea know-it-allThe European Union ha...,\n\nThe European Union (EU) haclassified AI s...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,...The Electronic Privacy Information Center (...,EPIC (Electronic Privacy Information Center)...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,sWhistleblowerand complaintplay a crucial role...,Whistleblowerand complaintplay a crucial rol...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",sThe UK'approach to AI idifferent from that of...,The UK'approach to data protection idifferen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",nobodyJapan'approach to AI regulation differf...,Japan'approach to AI regulation differfrom t...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",nobody knowthe answer to thiquestion.Foundati...,Foundational AI modelare the basic building ...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,nobody knowthe answer to thiquestion yet.Inte...,\n\nInternational cooperation on foundational...


### **Fine Tuning**

In [41]:
static_input = "[International Cooperation] Why is international cooperation on AI important?"
static_input = "International cooperation on AI is crucial due to the collaborative nature of AI research, innovation, and standardization. It maximizes the advantage of scale, focuses on responsible AI development, reduces barriers to innovation, aligns AI regulation, and addresses global challenges like climate change and pandemic preparedness."

def generate_prompt_with_control_code(control):
    output_texts = []
    for i in range(len(control['input'])):
        # Incorporate the static example with the actual Q&A for each item
        text = (f"### Example Question: {static_input}\n"
                f"### Example Answer: {static_input}\n"
                f"---\n"
                f"### Question: {control['input'][i]}\n"
                f"### Answer: {control['output'][i]}")
        output_texts.append(text)
    return output_texts

In [43]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Define a range of weight decay values to test
weight_decay_values = [0.0001, 0.001, 0.01, 0.1]

# Loop through each weight decay value
for wd in weight_decay_values:
    # Update the training parameters with the current weight decay value
    train_params = TrainingArguments(
        output_dir="./results_modified",
        num_train_epochs=5,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        save_steps=100,
        logging_steps=25,
        learning_rate=1e-4,
        weight_decay=wd,  # Update weight decay here
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="constant",
        report_to="tensorboard"
    )

    # Reinitialize your trainer with the new training arguments
    # Trainer
    fine_tuning = SFTTrainer(
        model=base_model,
        train_dataset=training_data,
        formatting_func=generate_prompt_with_control_code, #hereeee
        peft_config=peft_parameters,
        # dataset_text_field="text",
        tokenizer=llama_tokenizer,
        args=train_params
    )

    # Execute training
    fine_tuning.train()

    # Optionally evaluate the model here and log/print the results

    # Save the model for this iteration
    fine_tuning.model.save_pretrained(f"{refined_model}_wd_{wd}")


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.567800
50,0.871800
75,0.760200


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Step,Training Loss
25,1.569100
50,0.871800
75,0.760700


Step,Training Loss
25,1.569100
50,0.871800
75,0.760700


Step,Training Loss
25,1.569100
50,0.871800
75,0.760700


In [45]:
from tqdm.notebook import tqdm
import gc
generations_fine_tuned_prompt_engineer_controlcodes =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = fine_tuning.model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_fine_tuned_prompt_engineer_controlcodes.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations_fine_tuned_prompt_engineer_controlcodes


generating...:   0%|          | 0/19 [00:00<?, ?it/s]

['<s> ### Question: Why is international cooperation on AI important?\n ### Answer:  International cooperation on AI is important due to the collaborative nature of AI research, innovation, and standardization. It maximizes the advantage of scale, focuses on responsible AI development, reduces barriers to innovation, aligns AI regulation, and addresses global challenges like climate change and pandemic preparedness. It also aligns with the goals of the World Trade Organization (WTO) and the G7 to promote AI innovation and adoption',
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n ### Answer:  The research team took a multi-faceted approach to prepare for the interviews with academia and industry experts in the AI assessment. Firstly, they conducted a comprehensive literature review of existing research on AI, including academic papers, industry reports, and policy doc

In [46]:
df_test_all['fine_tuned_llama_generations_with_control_codes_prompting']=generations_fine_tuned_prompt_engineer_controlcodes


In [47]:

df_test_all['fine_tuned_llama_generations_with_control_codes_prompting']= df_test_all['fine_tuned_llama_generations_with_control_codes_prompting'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['fine_tuned_llama_generations_with_control_codes_prompting']= df_test_all['fine_tuned_llama_generations_with_control_codes_prompting'].apply(lambda x: x.replace('', '').replace('', '').replace('s', '').replace('', '').replace('s ', ''))
df_test_all

,input,output,prompt_engineering_control_codes_generations,original_llama_generations,fine_tuned_llama_generations_with_control_codes_prompting
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,nobody knowthe answer to thiquestion.Internat...,International cooperation on AI iimportant f...,International cooperation on AI i important ...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,"sure, here'how the research team prepared for ...",The research team took a multi-faceted appro...,The reearch team took a multi-faceted approa...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",t iimportant to note that the EU'AI Act istill...,\nCriticof the EU'AI Act have raised several ...,Critic argue that the EU' AI Act could tifle...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,nobody likea know-it-allThe European Union ha...,\n\nThe European Union (EU) haclassified AI s...,The European Union (EU) claifie AI ytem unde...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,...The Electronic Privacy Information Center (...,EPIC (Electronic Privacy Information Center)...,The Univeral Guideline for Artificial Intell...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,sWhistleblowerand complaintplay a crucial role...,Whistleblowerand complaintplay a crucial rol...,Whitleblower and complaint play a crucial ro...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",sThe UK'approach to AI idifferent from that of...,The UK'approach to data protection idifferen...,UK' approach i different from the European U...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",nobodyJapan'approach to AI regulation differf...,Japan'approach to AI regulation differfrom t...,Japan' approach to AI regulation differ from...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",nobody knowthe answer to thiquestion.Foundati...,Foundational AI modelare the basic building ...,Foundational AI model are the baic building ...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,nobody knowthe answer to thiquestion yet.Inte...,\n\nInternational cooperation on foundational...,International cooperation on foundational AI...


In [48]:
df_test_all.to_csv('prompt_engineering_control_codes_generations.csv')

## **Metrics**

In [27]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [28]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']], dtype=np.float32)

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.7259795 0.7777778 0.75      0.7777778]


In [29]:
raw_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['original_llama_generations']), axis=1)
raw_model_metrics = np.array(raw_model_metrics).mean(-1)
raw_model_metrics

array([0.00753218, 0.22768214, 0.04304937, 0.19757742], dtype=float32)

In [52]:
fine_tuned_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['fine_tuned_llama_generations_with_control_codes_prompting']), axis=1)
fine_tuned_model_metrics = np.array(fine_tuned_model_metrics).mean(-1)
fine_tuned_model_metrics

array([0.01454567, 0.25809222, 0.05973816, 0.21995728], dtype=float32)

In [55]:
df_metrics = pd.DataFrame([raw_model_metrics, fine_tuned_model_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2', 'Fine_tuned_llama2']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.007532,0.227682,0.043049,0.197577
Fine_tuned_llama2,0.014546,0.258092,0.059738,0.219957


In [50]:
prompt_engineering_templating_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['prompt_engineering_control_codes_generations']), axis=1)
prompt_engineering_templating_metrics = np.array(prompt_engineering_templating_metrics).mean(-1)
prompt_engineering_templating_metrics

array([0.00543408, 0.2874538 , 0.05192449, 0.24568899], dtype=float32)

In [56]:
df_metrics = pd.DataFrame([prompt_engineering_templating_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['prompt_engineering_control_codes_metrics']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
prompt_engineering_control_codes_metrics,0.005434,0.287454,0.051924,0.245689


In [58]:

df_metrics = pd.DataFrame([raw_model_metrics, fine_tuned_model_metrics, prompt_engineering_templating_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2', 'Fine_tuned_llama2_examples_prompting', 'prompt_engineering_control_codes_metrics']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.007532,0.227682,0.043049,0.197577
Fine_tuned_llama2_examples_prompting,0.014546,0.258092,0.059738,0.219957
prompt_engineering_control_codes_metrics,0.005434,0.287454,0.051924,0.245689
